I am getting this error:

```
RuleException:
ValueError in line 146 of /gpfs/gsfs6/users/MiegNCBI/ncbi_remap/prealn-wf/Snakefile:
The passed where expression: srr == SRR2176824 & srx == SRX1161016
            contains an invalid variable reference
            all of the variable refrences must be a reference to
            an axis (e.g. 'index' or 'columns'), or a data_column
            The currently defined references are: index,columns

  File "/gpfs/gsfs6/users/MiegNCBI/ncbi_remap/prealn-wf/Snakefile", line 146, in __rule_update_queue
  File "../lib/python/ncbi_remap/snakemake.py", line 60, in check_download
  File "../lib/python/ncbi_remap/io.py", line 105, in remove_id
  File "/data/fearjm/miniconda3/envs/ncbi_remap/lib/python3.5/site-packages/pandas/io/pytables.py", line 941, in remove
  File "/data/fearjm/miniconda3/envs/ncbi_remap/lib/python3.5/site-packages/pandas/io/pytables.py", line 4065, in delete
  File "/data/fearjm/miniconda3/envs/ncbi_remap/lib/python3.5/site-packages/pandas/io/pytables.py", line 4685, in __init__
  File "/data/fearjm/miniconda3/envs/ncbi_remap/lib/python3.5/site-packages/pandas/io/pytables.py", line 4710, in generate
  File "/data/fearjm/miniconda3/envs/ncbi_remap/lib/python3.5/concurrent/futures/thread.py", line 55, in run
  
```

I need to check the queue and make sure the query is working.

In [4]:
import sys
from tempfile import NamedTemporaryFile

import pandas as pd

sys.path.insert(0, '../../lib/python')
from ncbi_remap.io import add_data_columns

In [1]:
store = pd.HDFStore('../../sra.h5')
store.root.prealn.queue.table

/prealn/queue/table (Table(9190,), shuffle, blosc(9)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": StringCol(itemsize=10, shape=(2,), dflt=b'', pos=1)}
  byteorder := 'little'
  chunkshape := (2340,)
  autoindex := True
  colindexes := {
    "index": Index(6, medium, shuffle, zlib(1)).is_csi=False}

It looks like the srr/srx columns are no longer indexed.

In [5]:
store['prealn/queue'].head()

,srx,srr
873,SRX1968328,SRR3939099
874,SRX1968327,SRR3939098
875,SRX1968326,SRR3939097
876,SRX1968325,SRR3939096
877,SRX1968324,SRR3939095


I have two options: (1) re-build the queue with data_columns, (2) re-build the queue setting srx and srr as the index. I think (1) is the easiest solution because I have written all of the code to work with the queues the way they are. However, if I repack I will potentially run into this issue again. If I choose (2) I may break a bunch of stuff, but repacking should be ok.

Make a quick test and see how well things playout.

In [45]:
tmp = NamedTemporaryFile()
store_tmp = pd.HDFStore(tmp.name, mode='w')

In [46]:
df = store['prealn/queue']
df.set_index('srr', inplace=True)
df.head()

,srx
srr,
SRR3939099,SRX1968328
SRR3939098,SRX1968327
SRR3939097,SRX1968326
SRR3939096,SRX1968325
SRR3939095,SRX1968324


In [54]:
store_tmp.append('test/queue', df)

try:
    store_tmp['test/queue'].srr
except AttributeError:
    print('Yes using indexes screws up pulling out srr.')

Yes using indexes screws up code.


That did not work. I will use option (1) and just reset the data_columns.

### Do a little test

In [58]:
store_tmp.append('test/queue', df.reset_index(), append=False)
store_tmp['test/queue'].head()

,srr,srx
0,SRR3939099,SRX1968328
1,SRR3939098,SRX1968327
2,SRR3939097,SRX1968326
3,SRR3939096,SRX1968325
4,SRR3939095,SRX1968324


In [59]:
store_tmp.root.test.queue.table

/test/queue/table (Table(9190,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": StringCol(itemsize=10, shape=(2,), dflt=b'', pos=1)}
  byteorder := 'little'
  chunkshape := (2340,)
  autoindex := True
  colindexes := {
    "index": Index(6, medium, shuffle, zlib(1)).is_csi=False}

In [60]:
add_data_columns(store_tmp, 'test/queue', columns=['srx', 'srr'])

In [61]:
store_tmp.root.test.queue.table

/test/queue/table (Table(9190,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "srr": StringCol(itemsize=10, shape=(), dflt=b'', pos=1),
  "srx": StringCol(itemsize=10, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (2340,)
  autoindex := True
  colindexes := {
    "srx": Index(9, full, shuffle, zlib(1)).is_csi=True,
    "srr": Index(9, full, shuffle, zlib(1)).is_csi=True}

### Fix queue

In [7]:
add_data_columns(store, 'prealn/queue', columns=['srx', 'srr'])
store.root.prealn.queue.table

/prealn/queue/table (Table(9190,), shuffle, blosc(9)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "srx": StringCol(itemsize=10, shape=(), dflt=b'', pos=1),
  "srr": StringCol(itemsize=10, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (2340,)
  autoindex := True
  colindexes := {
    "srx": Index(9, full, shuffle, zlib(1)).is_csi=True,
    "srr": Index(9, full, shuffle, zlib(1)).is_csi=True}

In [65]:
store.close()